In [1]:
# Load packages
import tensorflow.keras
import pandas as pd
import numpy as np
import math
import glob
import os

In [2]:
# Compile the data

# Lie = 0 and Truth = 1

initialCSV = pd.concat(map(pd.read_csv, glob.glob(os.path.join('ICA', '*.csv'))), ignore_index = True)
    
initialCSV

,EEG.AF3,EEG.T7,EEG.Pz,EEG.T8,EEG.AF4,Truth
0,5.57764,-18.68978,0.12047,-11.03442,13.27478,1
1,12.78836,-43.83118,0.75530,-25.16123,31.04901,1
2,12.26007,-44.01917,1.04022,-24.15714,30.72010,1
3,12.64842,-46.38778,0.76016,-25.72193,32.60343,1
4,15.47116,-55.28282,0.98514,-31.43475,39.30939,1
...,...,...,...,...,...,...
518395,1.28386,-0.26978,-0.13770,-0.49923,-0.40874,0
518396,1.02784,-1.15967,0.04935,-0.17012,-0.45919,0
518397,0.76818,-0.51442,0.05882,0.40821,0.45476,0
518398,1.13579,0.75766,-0.66258,0.18601,1.03034,0


In [3]:
# Partition the data into 80% training data and 20% testing data

num_rows = len(initialCSV.index)
first80p = math.floor(0.8 * num_rows)
last20p = num_rows - first80p

x_train = initialCSV.head(first80p).drop(columns = ['Truth'])
y_train = initialCSV.head(first80p)[['Truth']]

x_test = initialCSV.tail(last20p).drop(columns = ['Truth'])
y_test = initialCSV.tail(last20p)[['Truth']]

x_train = x_train.to_numpy()
y_train = y_train.to_numpy()
x_test = x_test.to_numpy()
y_test = y_test.to_numpy()

print(x_train.shape)

x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))

print('Shape of x_train:\t' + str(x_train.shape))
print('Shape of y_train:\t' + str(y_train.shape))
print('Shape of x_test:\t' + str(x_test.shape))
print('Shape of y_test:\t' + str(y_test.shape))
print('Number of classes:\t' + str(np.max(y_train) - np.min(y_train) + 1))

(414720, 5)
Shape of x_train:	(414720, 1, 5)
Shape of y_train:	(414720, 1)
Shape of x_test:	(103680, 1, 5)
Shape of y_test:	(103680, 1)
Number of classes:	2


In [4]:
# One-Hot Encoding
# Transforms a scalar label to a k-dimensional vector
# Lie   = 0 = [ 1 , 0 ]
# Truth = 1 = [ 0 , 1 ]

def to_one_hot(y, num_class = 2):
    
    results = np.zeros((len(y), num_class))
    for i, label in enumerate(y): results[i, label] = 1.
    return results

y_train_vec = to_one_hot(y_train)
y_test_vec = to_one_hot(y_test)

print('Shape of y_train_vec:\t' + str(y_train_vec.shape))
print('Shape of y_test_vec:\t' + str(y_test_vec.shape))

Shape of y_train_vec:	(414720, 2)
Shape of y_test_vec:	(103680, 2)


In [5]:
# Randomly parition the training set into validation and non-validation sets

train_rows = len(y_train_vec)
train_80p = math.floor(0.8 * train_rows)

rand_indices = np.random.permutation(train_rows)
train_indices = rand_indices[0: train_80p]
valid_indices = rand_indices[train_80p: train_rows]

x_trn = x_train[train_indices, :]
y_trn = y_train_vec[train_indices, :]

x_val = x_train[valid_indices, :]
y_val = y_train_vec[valid_indices, :]

print('Shape of x_trn:\t\t' + str(x_trn.shape))
print('Shape of y_trn:\t\t' + str(y_trn.shape))
print('Shape of x_val:\t\t' + str(x_val.shape))
print('Shape of y_val:\t\t' + str(y_val.shape))

Shape of x_trn:		(331776, 1, 5)
Shape of y_trn:		(331776, 2)
Shape of x_val:		(82944, 1, 5)
Shape of y_val:		(82944, 2)


In [6]:
# Build the model

from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers

model = models.Sequential()

# Convolutional layers
model.add(layers.Conv1D(256, 1, activation = 'relu', input_shape = (1, 5)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling1D(pool_size = 2, strides = 1, padding = 'same'))
model.add(layers.Dropout(0.25))
model.add(layers.Conv1D(128, 1, activation = 'relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling1D(pool_size = 2, strides = 1, padding = 'same'))
model.add(layers.Dropout(0.25))
model.add(layers.Conv1D(64, 1, activation = 'relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling1D(pool_size = 2, strides = 1, padding = 'same'))
model.add(layers.Dropout(0.25))

# Fully-connected layers
model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(128, activation = 'relu'))
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(2, activation = 'softmax'))

model.summary()

c:\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 1, 256)         │         1,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1, 256)         │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 1, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 1, 128)         │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 1, 128)         │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 1, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 1, 64)          │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 1, 64)          │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 1, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │        16,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 102,402 (400.01 KB)

 Trainable params: 101,506 (396.51 KB)

 Non-trainable params: 896 (3.50 KB)

In [7]:
# Define model optimizer and loss function

model.compile(
    optimizers.Adam(learning_rate = 0.001),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [8]:
# Train the model and store parameters and loss values

history = model.fit(x_trn, y_trn, batch_size = 128, epochs = 50, validation_data = (x_val, y_val))

Epoch 1/50
2592/2592 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.5334 - loss: 0.6881 - val_accuracy: 0.5428 - val_loss: 0.6844
Epoch 2/50
2592/2592 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.5436 - loss: 0.6839 - val_accuracy: 0.5419 - val_loss: 0.6842
Epoch 3/50
2592/2592 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.5451 - loss: 0.6828 - val_accuracy: 0.5446 - val_loss: 0.6834
Epoch 4/50
2592/2592 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.5473 - loss: 0.6824 - val_accuracy: 0.5439 - val_loss: 0.6817
Epoch 5/50
2592/2592 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.5477 - loss: 0.6816 - val_accuracy: 0.5462 - val_loss: 0.6816
Epoch 6/50
2592/2592 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.5468 - loss: 0.6813 - val_accuracy: 0.5444 - val_loss: 0.6824
Epoch 7/50
2592/2592 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.5480 - loss: 0.6809 - val_accuracy: 0.5439 - val_loss: 0.6826
Epoch 8/50
2592/2592 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.5480 - loss: 0.6808 -

In [9]:
model.save('seq18.keras')

# Evaluate testing accuracy on the testing dataset 

loss_and_acc = model.evaluate(x_test, y_test_vec)

3240/3240 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.3702 - loss: 0.7160


In [10]:
# Test single data points
# Lie   = 0 = [ 1 , 0 ]
# Truth = 1 = [ 0 , 1 ]

# Test a lie -- Subject 18, Session 2, Point 242

test_lie = np.array((-9.62814, -7.06874, -3.28544, 1.90316, -130.425)).astype(np.float32)
test_lie = np.reshape(test_lie, (1, 1, 5))

pred_lie = model.predict(test_lie)
print(pred_lie[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
[0.7267138 0.2732863]


In [11]:
# Test a truth -- Subject 23, Session 1, Point 744

test_truth = np.array((12.66325, 57.29564, 14.44304, 45.30457, -9.97659)).astype(np.float32)
test_truth = np.reshape(test_truth, (1, 1, 5))

pred_truth = model.predict(test_truth)
print(pred_truth[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
[0.33643445 0.66356564]


In [12]:
# Unit Testing

import unittest 

class TestModel(unittest.TestCase):
    
    def test_lie_lie(self):
        self.assertGreater(pred_lie[0][0], 0)
        self.assertLess(pred_lie[0][0], 1)
        
    def test_lie_truth(self):
        self.assertGreater(pred_lie[0][1], 0)
        self.assertLess(pred_lie[0][1], 1)
        
    def test_truth_lie(self):
        self.assertGreater(pred_truth[0][0], 0)
        self.assertLess(pred_truth[0][0], 1)
        
    def test_truth_truth(self):
        self.assertGreater(pred_truth[0][1], 0)
        self.assertLess(pred_truth[0][1], 1)

unittest.main(argv = [''], verbosity = 2, exit = False)

test_lie_lie (__main__.TestModel.test_lie_lie) ... ok
test_lie_truth (__main__.TestModel.test_lie_truth) ... ok
test_truth_lie (__main__.TestModel.test_truth_lie) ... ok
test_truth_truth (__main__.TestModel.test_truth_truth) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.003s

OK
